<a href="https://colab.research.google.com/github/nahyunryou/ML_Project/blob/main/Catboost3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 18 kB/s 


In [2]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 348 kB 12.1 MB/s 
     |████████████████████████████████| 81 kB 10.8 MB/s 
     |████████████████████████████████| 209 kB 77.4 MB/s 
     |████████████████████████████████| 78 kB 7.9 MB/s 
     |████████████████████████████████| 147 kB 86.1 MB/s 
     |████████████████████████████████| 50 kB 797 kB/s 
     |████████████████████████████████| 112 kB 61.7 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=aef232943c194aaf3a1072e694c5e25df3340161051eec0ddc58b8a541d245d9
  Stored in directory: /root/.cache/pip/wheels/7f/1a/65/84ff8c386bec21fca6d220ea1f5498a0367883a78dd5ba6122
Successfully built pyperclip


In [3]:
import warnings
warnings.filterwarnings('ignore')

import glob
import pandas as pd
import numpy as np

from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold,train_test_split
from sklearn.metrics import log_loss
from sklearn.preprocessing import OneHotEncoder
import random

import optuna
from optuna.samplers import TPESampler

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
train = pd.read_csv('/content/drive/MyDrive/ml_project/processed_train.csv')
#train = train.drop(['index'], axis=1)
train.fillna('NAN',inplace=True)

test = pd.read_csv('/content/drive/MyDrive/ml_project/processed_test.csv')
#test = test.drop(['index'],axis=1)
test.fillna('NAN',inplace=True)

submit = pd.read_csv('/content/drive/MyDrive/ml_project/sample_submission.csv')

In [6]:
edu_dict = {'Lower secondary':1,
 'Secondary / secondary special':2,
 'Incomplete higher':3,
 'Higher education':4,
 'Academic degree':5}

train['edu_type'].replace(edu_dict,inplace=True)
test['edu_type'].replace(edu_dict,inplace=True)

In [7]:
house_dict = {'With parents':3,
 'Co-op apartment':2,
 'Municipal apartment':1,
 'Rented apartment':4,
 'Office apartment':5,
 'House / apartment':6}

train['house_type'].replace(house_dict,inplace=True)
test['house_type'].replace(house_dict,inplace=True)

In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23392 entries, 0 to 23391
Data columns (total 28 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         23392 non-null  int64  
 1   gender             23392 non-null  object 
 2   car                23392 non-null  object 
 3   reality            23392 non-null  object 
 4   income_total       23392 non-null  float64
 5   income_type        23392 non-null  object 
 6   edu_type           23392 non-null  int64  
 7   family_type        23392 non-null  object 
 8   house_type         23392 non-null  int64  
 9   work_phone         23392 non-null  int64  
 10  home_phone         23392 non-null  int64  
 11  email              23392 non-null  int64  
 12  occup_type         23392 non-null  object 
 13  family_size        23392 non-null  float64
 14  begin_month        23392 non-null  int64  
 15  credit             23392 non-null  float64
 16  days_unemployed    233

#Training

In [9]:
#Optuna용 Train셋
X = train.drop(['credit'],axis=1)
y = train['credit']
X_test = test.copy()

In [10]:
def objective(trial):
  param = {
      "random_state":42,
      'learning_rate' : trial.suggest_loguniform('learning_rate', 0.01, 0.3),
      'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 0.01, 100.00),
      "n_estimators":trial.suggest_int("n_estimators", 1000, 10000),
      "max_depth":trial.suggest_int("max_depth", 4, 16),
      'random_strength' :trial.suggest_int('random_strength', 0, 100),
      "colsample_bylevel":trial.suggest_float("colsample_bylevel", 0.4, 1.0),
      "l2_leaf_reg":trial.suggest_float("l2_leaf_reg",1e-8,3e-5),
      "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
      "max_bin": trial.suggest_int("max_bin", 200, 500),
      'od_type': trial.suggest_categorical('od_type', ['IncToDec', 'Iter']),
  }

  X_train, X_valid, y_train, y_valid = train_test_split(X,y,test_size=0.2)
  
  # cat_features =[0,1,2,5,6,7,8,15]
  cat_features =[1,2,3,5,7,12]
  cat = CatBoostClassifier(**param)
  cat.fit(X_train, y_train,
          eval_set=[(X_train, y_train), (X_valid,y_valid)],
          early_stopping_rounds=35,cat_features=cat_features,
          verbose=100)
  cat_pred = cat.predict_proba(X_valid)
  log_score = log_loss(y_valid, cat_pred)

  return log_score

In [11]:
sampler = TPESampler(seed=42)
study = optuna.create_study(
    study_name = 'cat_parameter_opt',
    direction = 'minimize',
    sampler = sampler,
)
study.optimize(objective, n_trials=10)
print("Best Score:",study.best_value)
print("Best trial",study.best_trial.params)

[I 2022-12-08 11:56:56,740] A new study created in memory with name: cat_parameter_opt


0:	learn: 1.0821244	test: 1.0821244	test1: 1.0818482	best: 1.0818482 (0)	total: 59.8ms	remaining: 7m 33s
100:	learn: 0.8711219	test: 0.8711219	test1: 0.8645644	best: 0.8645644 (100)	total: 1.15s	remaining: 1m 24s
200:	learn: 0.8709316	test: 0.8709316	test1: 0.8642077	best: 0.8642077 (200)	total: 2.15s	remaining: 1m 18s
300:	learn: 0.8709314	test: 0.8709314	test1: 0.8642032	best: 0.8642032 (300)	total: 3.16s	remaining: 1m 16s
400:	learn: 0.8709314	test: 0.8709314	test1: 0.8642031	best: 0.8642031 (400)	total: 4.33s	remaining: 1m 17s
500:	learn: 0.8709314	test: 0.8709314	test1: 0.8642031	best: 0.8642031 (500)	total: 5.58s	remaining: 1m 18s
600:	learn: 0.8709314	test: 0.8709314	test1: 0.8642031	best: 0.8642031 (600)	total: 6.61s	remaining: 1m 16s
700:	learn: 0.8709314	test: 0.8709314	test1: 0.8642031	best: 0.8642031 (686)	total: 7.6s	remaining: 1m 14s


[I 2022-12-08 11:57:05,138] Trial 0 finished with value: 0.8642030816748936 and parameters: {'learning_rate': 0.03574712922600244, 'bagging_temperature': 63.512210106407046, 'n_estimators': 7588, 'max_depth': 11, 'random_strength': 15, 'colsample_bylevel': 0.49359671220172163, 'l2_leaf_reg': 1.7519275289243016e-06, 'min_child_samples': 88, 'max_bin': 380, 'od_type': 'IncToDec'}. Best is trial 0 with value: 0.8642030816748936.


Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.8642030817
bestIteration = 723

Shrink model to first 724 iterations.
0:	learn: 0.9867939	test: 0.9867939	test1: 0.9873989	best: 0.9873989 (0)	total: 41.7ms	remaining: 2m 1s
100:	learn: 0.6749697	test: 0.6749697	test1: 0.7743979	best: 0.7743979 (100)	total: 4.14s	remaining: 1m 55s
200:	learn: 0.5852068	test: 0.5852068	test1: 0.7679011	best: 0.7679011 (200)	total: 8.73s	remaining: 1m 57s


[I 2022-12-08 11:57:15,488] Trial 1 finished with value: 0.7679011389674975 and parameters: {'learning_rate': 0.2708160864249968, 'bagging_temperature': 21.368329072358772, 'n_estimators': 2911, 'max_depth': 6, 'random_strength': 18, 'colsample_bylevel': 0.5825453457757226, 'l2_leaf_reg': 1.5747445384650815e-05, 'min_child_samples': 46, 'max_bin': 287, 'od_type': 'IncToDec'}. Best is trial 1 with value: 0.7679011389674975.


...

0:	learn: 1.0109841	test: 1.0110510	test1: 1.0112485	best: 1.0112485 (0)	total: 49.9ms	remaining: 1m 43s
100:	learn: 0.2340707	test: 0.3047717	test1: 0.7749435	best: 0.7540147 (83)	total: 42.3s	remaining: 13m 46s


[I 2022-12-08 12:43:23,605] Trial 8 finished with value: 0.7540147432020732 and parameters: {'learning_rate': 0.20441878352493792, 'bagging_temperature': 0.7742116473996251, 'n_estimators': 2076, 'max_depth': 13, 'random_strength': 76, 'colsample_bylevel': 0.7367663185416977, 'l2_leaf_reg': 2.3131305726837285e-05, 'min_child_samples': 52, 'max_bin': 357, 'od_type': 'IncToDec'}. Best is trial 7 with value: 0.7118148655460776.


Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.7540147432
bestIteration = 83

Shrink model to first 84 iterations.
0:	learn: 1.0918802	test: 1.0919526	test1: 1.0916815	best: 1.0916815 (0)	total: 108ms	remaining: 12m 4s
100:	learn: 0.8734488	test: 0.8749738	test1: 0.8640089	best: 0.8640089 (100)	total: 5.85s	remaining: 6m 23s
200:	learn: 0.8468110	test: 0.8493436	test1: 0.8381827	best: 0.8381827 (200)	total: 13.1s	remaining: 7m 4s
300:	learn: 0.8379628	test: 0.8416063	test1: 0.8311924	best: 0.8311924 (300)	total: 18.1s	remaining: 6m 26s
400:	learn: 0.8311093	test: 0.8359138	test1: 0.8271370	best: 0.8271370 (400)	total: 23.5s	remaining: 6m 10s
500:	learn: 0.8212932	test: 0.8275024	test1: 0.8206294	best: 0.8206294 (500)	total: 29.3s	remaining: 6m 4s
600:	learn: 0.8128869	test: 0.8204235	test1: 0.8153032	best: 0.8153032 (600)	total: 35.5s	remaining: 6m 1s
700:	learn: 0.8028228	test: 0.8114890	test1: 0.8079008	best: 0.8078994 (699)	total: 41.3s	remaining: 5m 55s
800:	le

[I 2022-12-08 12:47:18,906] Trial 9 finished with value: 0.7199496874249156 and parameters: {'learning_rate': 0.01443340240633889, 'bagging_temperature': 0.0133572404119741, 'n_estimators': 6728, 'max_depth': 8, 'random_strength': 51, 'colsample_bylevel': 0.9445398843556558, 'l2_leaf_reg': 7.486273952174759e-06, 'min_child_samples': 44, 'max_bin': 427, 'od_type': 'IncToDec'}. Best is trial 7 with value: 0.7118148655460776.


Best Score: 0.7118148655460776
Best trial {'learning_rate': 0.012863908101989912, 'bagging_temperature': 0.27155819552829413, 'n_estimators': 2042, 'max_depth': 15, 'random_strength': 62, 'colsample_bylevel': 0.5985388149115896, 'l2_leaf_reg': 1.9161149250778487e-06, 'min_child_samples': 34, 'max_bin': 297, 'od_type': 'IncToDec'}


In [12]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
folds = []
for train_idx, valid_idx in skf.split(train, train['credit']):
  folds.append((train_idx,valid_idx))

In [13]:
random.seed(42)
cat_models={}

cat_features =[1,2,3,5,7,12]

for fold in range(5):
  print(f'===================================={fold+1}============================================')
  train_idx, valid_idx = folds[fold]
  X_train = train.drop(['credit'],axis=1).iloc[train_idx].values 
  X_valid = train.drop(['credit'],axis=1).iloc[valid_idx].values
  y_train = train['credit'][train_idx].values
  y_valid = train['credit'][valid_idx].values

  cat = CatBoostClassifier(**study.best_params)
  cat.fit(X_train, y_train,
          eval_set=[(X_train, y_train), (X_valid,y_valid)],
          early_stopping_rounds=35,cat_features=cat_features,
          verbose=100)
  cat_models[fold] = cat
  print(f'================================================================================\n\n')

====================================1============================================
0:	learn: 1.0925701	test: 1.0925701	test1: 1.0925692	best: 1.0925692 (0)	total: 18.4ms	remaining: 37.5s
100:	learn: 0.8754531	test: 0.8782147	test1: 0.8803563	best: 0.8803563 (100)	total: 30.9s	remaining: 9m 54s
200:	learn: 0.8381615	test: 0.8459430	test1: 0.8510435	best: 0.8510435 (200)	total: 1m 19s	remaining: 12m 11s
300:	learn: 0.8269867	test: 0.8370388	test1: 0.8434331	best: 0.8434331 (300)	total: 1m 33s	remaining: 8m 59s
400:	learn: 0.8145027	test: 0.8280194	test1: 0.8366572	best: 0.8366572 (400)	total: 2m 6s	remaining: 8m 35s
500:	learn: 0.8018909	test: 0.8195498	test1: 0.8303443	best: 0.8303425 (499)	total: 2m 39s	remaining: 8m 10s
600:	learn: 0.7968975	test: 0.8162750	test1: 0.8283738	best: 0.8283728 (599)	total: 2m 58s	remaining: 7m 8s
700:	learn: 0.7885293	test: 0.8101881	test1: 0.8240702	best: 0.8240702 (700)	total: 3m 23s	remaining: 6m 28s
800:	learn: 0.7765504	test: 0.8016006	test1: 0.817921

#Test

In [14]:
submit.iloc[:,1:]=0
for fold in range(5):
  submit.iloc[:,1:] += cat_models[fold].predict_proba(test)/5

In [15]:
import datetime
now = datetime.datetime.now()
created_time = now.strftime('%m%d-%H%M')

submit.to_csv(f'/content/drive/MyDrive/ml_project/{created_time}_submit.csv',index=False)

In [16]:
submit.head(20)

,index,0,1,2
0,26457,0.049213,0.141508,0.809279
1,26458,0.311101,0.112029,0.576870
2,26459,0.054821,0.155063,0.790116
3,26460,0.045511,0.079300,0.875189
4,26461,0.083925,0.174212,0.741863
5,26462,0.067856,0.255505,0.676638
6,26463,0.198667,0.758700,0.042633
7,26464,0.103434,0.175930,0.720636
8,26465,0.043730,0.086594,0.869676
9,26466,0.030617,0.190771,0.778611
